In [0]:
# 03_silver_claims_enriched.ipynb
# SOURCE:  kardia_silver.silver_claims_current + kardia_silver.silver_providers_dim
# OUTPUT:  kardia_silver.silver_claims_enriched  (snapshot join on ProviderID)
# TRIGGER: Joins current claims with current provider attributes for Gold KPIs.

CLAIMS_TABLE = "kardia_silver.silver_claims_current"
PROV_DIM     = "kardia_silver.silver_providers_dim"
ENRICH_TABLE = "kardia_silver.silver_claims_enriched"

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS kardia_silver")

claims_df  = spark.table(CLAIMS_TABLE)
prov_df    = (
    spark.table(PROV_DIM)
         .filter("is_current")
         .select("ProviderID", "ProviderSpecialty", "ProviderLocation")
)

In [0]:
enriched_df = claims_df.join(prov_df, on="ProviderID", how="left")

if not spark.catalog.tableExists(ENRICH_TABLE):
    (enriched_df.limit(0).write
                         .format("delta")
                         .saveAsTable(ENRICH_TABLE))

(
    enriched_df.write
               .format("delta")
               .mode("overwrite")
               .option("overwriteSchema", "true")
               .option("mergeSchema", "true") 
               .saveAsTable(ENRICH_TABLE)
)

print("Enriched snapshot overwritten.")